# Motivating example

In [ ]:
import numpy as np
import numba

In [ ]:
def mean_python(a):
    total = 0
    for i in a:
        total += i
    return total / len(a)

def mean_numpy(a):
    return np.mean(a)

mean_numba = numba.jit(mean_python)

Here, I'm using `numba.njit` as a function, but it's more common to see it used as a decroator. `mean_numba = numba.jit(mean_python)` is equivalent to:

```python
@numba.jit
def mean_numba(a):
    total = 0
    for i in a:
        total += i
    return total / len(a)
```

In [ ]:
a = np.random.randint(1, 100, 1000000)

In [ ]:
%time mean_python(a)

In [ ]:
%time mean_numpy(a)

In [ ]:
%time mean_numba(a)

In [ ]:
%timeit mean_python(a)
%timeit mean_numpy(a)
%timeit mean_numba(a)

# Why is numba fast

For every instance of an object python has to figure out what type it is and how to perform the operation Numpy is able to skip all this by knowing the container only has one kind of object in it. This way it can just dispach to compiled code which performs a defined set of operations on those objects. Numba is able to look at python code and figure out what the fast cpu code would be. In practice, this means loops and flow control are no longer slow.

# Trying it out

For a first go round, let's try extending the previous example.

Something we often want to calculate along with the mean is the [variance](https://en.wikipedia.org/wiki/Variance).

$$ Var(X) = (\frac{1}{n}\sum_{i=1}^n x_i^2) - \mu^2$$

Where $\mu$ is the mean:

$$ \mu = \frac{1}{n}\sum_{i=1}^2 x_i $$

Without taking using `np.mean` or `np.var`, given a two dimensional numpy array, calculate the mean and variance for each row in a single pass:

In [ ]:
@numba.njit
def meanvar(x):
    # your code here
    return mean, var

In [ ]:
a = np.random.normal(1, 3, (100000))

In [ ]:
# Test that should pass:
np.allclose(meanvar(a), (np.mean(a), np.var(a)))

In [ ]:
%timeit meanvar(a)

In [ ]:
%timeit np.mean(a), np.var(a)

## A bit more about `njit` and `jit`

So what was this magic decorator that made the code so fast? It's a function which looks over the python byte code and translates it to LLVM IR. It [has a few options](https://numba.pydata.org/numba-doc/dev/user/jit.html) which modify how this is done:

### `nopython=True`

When you use the bare `@jit` decorator, numba will try to speed up the function, but won't throw an error if it fails. Passing `nopython=True` ensures that your function will be compilled, and is probably what you want. This is recommended so strongly that the also provide `njit` – which is equivalent to `@jit` with `nopython=True`. In the future, `nopython=True` will become the default for `@jit`.

### `cache`

As you saw above, numba only compiles a function once it is called. Unfortunatley, this means that everytime you restart python numba will need to compile a function again when it's called. Passing `cache=True` makes numba cache the compilled function to disk, so it doesn't need to compile again. This is useful for complicated functions, which can have compile times longer than their run time.

### `parallel`

A pretty cool option. If you pass this `numba` will try and make your code run across multiple threads. This is covered more by one of the challenges.

### Explicit types (eager compilation)

If you know the types you want to compile a method for, you can specify them by passing `numba` a type signature string. This is out of scope for todays workshop, but you can find out more [here](https://numba.pydata.org/numba-doc/dev/user/jit.html#eager-compilation)